# Particle Fluid

Modeling fluid motion by simulating a set of particles. 

Credit to Professor Bo Zhu, on whose Dartmouth COSC89 course notes the following code is based

### Structure: 
Give each particle an index\
each particle has a position (2D vector), velocity (2D vector), mass (scalar), density (scalar), pressure (scalar)\
Have a global constant h (kernel radius)

Have kernels to calculate w_spiky, gradient_w_spiky, laplacian_w_spiky\
Where w_spiky takes two indices (which it uses to calculate the radius r)\
grad and laplacian take just one index (which it uses to find the velocity of that index)\

Have kernels to calculate w_vis, gradient_w_vis, laplacian_w_vis\
Same inputs as for the corresponding spiky kernels

Spatial Hashing: \
Keep a dictionary hashing a vec2 -> list of indices\
Iterate over all points and add them to the array that corresponds 

Have each particle keep track of its neighbors with an array of arrays

Use explicit time integration to advance velocity and position for each time step

Have methods to update:\
Neighbors\
Density\
Pressure\
Pressure Force\
Viscosity Force\
Body Force\
Boundary Collision Force (penalty based collision respoonse)\
Temperature and Buoyancy (?)

In [2]:
import taichi as ti
import taichi.math as tm

ti.init()

### Global variables ###
dt = 2e-3
num_particles = 100
substeps = 10
screen_size = 800
ks = 2e2
kd = 0.5e1
radius = 5
h = 5 # prof's uses 0.8, but unsure what those units are
bowl_r = 500
c = tm.vec2(0.5, 0.7) # center of the bowl

pos = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
vel = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
force = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
r = ti.field(dtype=ti.f32, shape=num_particles)
m = ti.field(dtype=ti.f32, shape=num_particles)
voxels = {}

### Spiky Kernel Functions ###
@ti.kernel
def w_spiky(idx1:ti.i32, idx2: ti.i32) -> ti.f32: # change this to pass in a vector?
    valToReturn = 0
    r = (pos[idx2] - pos[idx1]).norm()
    if (r > 0 and r <= h):
        valToReturn = 15.0 / (math.pi * ti.pow(h, 6)) * ti.pow(h - r, 3)
    return valToReturn

@ti.kernel
def gradient_w_spiky(idx1:ti.i32, idx2: ti.i32) -> tm.vec2: 
    valToReturn = tm.vec2(0,0)
    v = pos[idx2] - pos[idx1] # should v be velocity or the displacement vector
    r = v.norm() 
    if (r > 0 and r <= h):
        valToReturn = -45.0 / (math.pi * ti.pow(h, 6)) * ti.pow (h - r, 2) * v / r
    return valToReturn

@ti.kernel
def laplacian_w_spiky(idx1:ti.i32, idx2: ti.i32) -> ti.f32:
    valToReturn = 0
    v = pos[idx2] - pos[idx1] # should v be velocity or the displacement vector
    r = v.norm() 
    if (r > 0 and r <= h):
        valToReturn = -90.0 / (math.pi * ti.pow(h, 6)) * (h - r) * (h - 2*r) / r
    return valToReturn


### Viscosity Kernel Functions ###
@ti.kernel
def w_vis(idx1:ti.i32, idx2: ti.i32) -> ti.f32: # do I need this
    valToReturn = 0
    r = (pos[idx2] - pos[idx1]).norm()
    if (r > 0 and r <= h):
        valToReturn = 15.0 / (2 * math.pi * ti.pow(h, 3)) * (-r / (2 * ti.pow(h, 3)) + ti.pow(r, 2)/ti.pow(h, 2) + h / (2 * r) - 1)
    return valToReturn

@ti.kernel
def gradient_w_vis(idx1:ti.i32, idx2: ti.i32) -> tm.vec2: # do I need this
    valToReturn = tm.vec2(0,0)
    v = pos[idx2] - pos[idx1]
    r = v.norm()
    if (r > 0 and r <= h):
        valToReturn = 15.0 / (2 * math.pi * ti.pow(h, 3)) * (-3 * r / (2 * ti.pow(h, 3)) + 2 / ti.pow(h, 2) - h/ (2 * ti.pow(r, 3))) * v
    return valToReturn

@ti.kernel
def laplacian_w_vis(idx1:ti.i32, idx2: ti.i32) -> ti.f32:
    valToReturn = 0
    v = pos[idx2] - pos[idx1]
    r = v.norm()
    if (r > 0 and r <= h):
        valToReturn = 45.0 / (math.pi * ti.pow(h, 6)) * (h - r)
    return valToReturn


### Spatial Hashing Functions ###
# @param - index of the point being added to voxels
@ti.func
def add_point(idx:ti.i32):
    voxelX = ti.floor(pos[idx][0] * screen_size / h)
    voxelY = ti.floor(pos[idx][1] * screen_size / h)
    if (voxelX, voxelY) not in voxels:
        voxels[(voxelX, voxelY)] = []
    voxels.get((voxelX, voxelY)).append(idx)


@ti.kernel
def update_voxels():
    voxels.clear()
    for idx in range(num_particles):
        add_point(idx)


### Simulator Functions ###
@ti.func
def clear_force():
    # right now, not really clearing force, but rather clearing it out and adding gravity
    for idx in range(num_particles):
        force[idx] = tm.vec2(0, -1)

@ti.func
def update_neighbors():
    pass

@ti.func
def update_density():
    pass

@ti.func
def update_pressure():
    pass

@ti.func
def update_pressure_force():
    pass

@ti.func
def update_viscosity_force():
    pass

@ti.func
def update_body_force():
    pass

@ti.func
def update_boundary_collision_force():
    for i in range(num_particles):
        # boundary check with the bowl
        phi = bowl_r - ((pos[i] - c).norm() * screen_size) - r[i]
        neg_grad = (pos[i] - c).normalized()
        if phi < 0: # outside of the bowl
            force[i] += ks * (phi) * neg_grad
            force[i] += kd * (ti.Vector([0, 0]) - vel[i]).dot(neg_grad) * neg_grad

@ti.kernel
def substep():
    clear_force()
    update_neighbors()
    update_density()
    update_pressure()
    update_pressure_force()
    update_viscosity_force()
    update_body_force()
    update_boundary_collision_force()
    # update position and velocity
    for idx in range(num_particles):
        vel[idx] = vel[idx] + dt * force[idx] / m[idx]
        pos[idx] = pos[idx] + dt * vel[idx]

        
### Setup ###
@ti.kernel
def init_particles():
    sqrt = ti.floor(ti.sqrt(num_particles), ti.i32)
    disp = 0.5/sqrt
    for idx in range(num_particles):
        r[idx] = radius
        vel[idx] = tm.vec2(0, 0)
        force[idx] = tm.vec2(0, 0)
        m[idx] = 1.0
        pos[idx] = tm.vec2(0.25 + disp * (ti.floor(idx/ sqrt)), 0.25 + disp * (idx % sqrt))    


### Driver ###
def main():
    gui = ti.GUI('Particle Fluid', (screen_size, screen_size))
        
    init_particles()
        
    while gui.running:
        pass
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = pos.to_numpy()
        for i in range(num_particles):
            gui.circle(pos=X[i], radius = r[i])
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, osx, python 3.9.7
[I 10/11/22 14:48:54.553 1544189] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=x64
